# This file is to finish the udacity nano degree problem for Anna_KaRNNa with Pytorch

This file is to build a chracter-wise RNN trained on Anna Kerenia.

## 1. Load Packages

In [305]:
import time
from collections import namedtuple
import numpy as np
import torch

## 2. Load Data

In [306]:
with open('anna.txt','r') as f:
    text = f.read()
vocab =sorted(set(text))
vocab_to_int = {c: i for i,c in enumerate(vocab)}
int_to_vocab =dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text])

One of the best first line of a book ever.

In [307]:
lr = 0.001

In [308]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

## 3. Make training mini-batches

In [309]:
# Follow the udacity's books function to create the batch-iterator here

In [310]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns mini-batches of size
       n_seqs x n_steps from arr.
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size]
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [311]:
# def get_batches(arr, batch_size, n_steps):
#     ''' Create generator that returns the batches of size 
#         batch_size * n_steps
#         Here we follow the udacity's notebook's rule, we will remove the last piece
#         of the samples smaller than the batch_size.
#     '''
#     size = batch_size * n_steps
#     n_batches = len(arr)/size
    
#     # Here we only keep enough numbers 
#     arr = arr[:size*n_batches]
    
#     # reshape batches into batch_size row
#     arr = arr.reshape((batch_size,-1))
    
#     for n in range(0,arr.shape[1],n_steps):
#         x = arr[:,n:n+n_steps]
#         y_temp = arr[:,n+1:n+n_steps+1]
        
#         # for the last batch, the y will be short with one.
#         # we will pad the last value with 0
        
#         y = np.zeros(x.shape)
#         y[:,:y_temp.shape[1]]=y_temp
        
#         yield x,y

In [312]:
n_seqs = 128
n_steps = 100
label_size = len(vocab)

In [313]:
print('x \n', x[:10, :10])
print('\n y \n', y[:10, :10])

('x \n', tensor([[[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 1.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0., 

## 4. Model Building

In [314]:
# Here we build the LSTM neural network models.

In [315]:
## Need to rewrite this function

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class LSTM_Char(nn.Module):
    
    def __init__(self,hidden_dim,n_steps,n_layers,label_size,batch_size,drop_out=0.5,lr=0.001):
        
        super(LSTM_Char,self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.label_size = label_size
        self.n_layers = n_layers
        self.drop_out = drop_out
        self.dropout = nn.Dropout(drop_out)
        self.lstm = nn.LSTM(input_size= label_size ,hidden_size = hidden_dim,num_layers = n_layers,\
                            dropout=drop_out,batch_first=True)
        self.hidden = self.init_hidden(n_steps)
        self.out = nn.Linear(hidden_dim,label_size)
        self.init_weight()
        
    def init_hidden(self,n_seqs):
        ## Two layer of hidden, first is for the input x, second is for the hidden state
        weight = next(self.parameters()).data
        return (Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()),
                Variable(weight.new(self.n_layers,n_seqs,self.hidden_dim).zero_()))
    
    def forward(self,x,hc):
        # Here we put the hc in the tuple
        x,(h,c) = self.lstm(x,hc)
        x = self.dropout(x)
        
        # Put all the LSTM outputs together
        x = x.view(x.size()[0]*x.size()[1],self.hidden_dim)
        
        x = self.out(x)
        
        return x,(h,c)
    
    def predict(self,char,h=None,cuda=False,top_k=None):
        ''' Given a character, predict the next character.
            Return the predicted character and the hidden state.
        '''
        # Currently we don't consider cuda here.
        if h is None:
            h=self.init_hidden(1)
        x = np.array([[vocab_to_int[char]]])

        x = one_hot_encode(x,self.label_size)
        
        inputs = Variable(torch.from_numpy(x),volatile = True)
        
        h = tuple(Variable(each.data,volatile = True) for each in h)
        
        out,h= self.forward(inputs,h)
        
        p = F.softmax(out).data
        
        if top_k is None:
            top_ch = np.arange(self.label_size)
        else:
            p,top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
            
        p = p.numpy().squeeze()

        char = np.random.choice(top_ch, p=p/p.sum())

        return int_to_vocab[char],h
    
    def init_weight(self):
        initrange = 0.1
        #set bias tensor to all zeros
        self.out.bias.data.fill_(0)
        # set weight to randeom uniform
        self.out.weight.data.uniform_(-1,1)
        

In [316]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((arr.shape[0],arr.shape[1], n_labels))
    
    return one_hot

## 5. Model Training

In [317]:
net = LSTM_Char(hidden_dim = 512,n_steps=100,\
                n_layers = 2,label_size = len(vocab),\
                batch_size=128)

In [318]:
net.train()
opt = torch.optim.Adam(net.parameters(),lr=lr)
criterion = nn.CrossEntropyLoss()

In [319]:
# Here we can do the split between training and test data
val_frac = 0.2
val_index = int(len(encoded)*(1-val_frac))
data,val_data = encoded[:val_index],encoded[val_index:]

## 6. Check Results

In [ ]:
epochs = 10
clip = 5
counter = 0
print_every = 10
for e in range(epoch):
    h = net.init_hidden(n_seqs)
    for x, y in get_batches(data,n_seqs,n_steps):
        counter += 1
        
        # Need to do one-hot encoding and make them the torch tensors
        x = one_hot_encode(x,label_size)
        x, y =torch.from_numpy(x),torch.from_numpy(y)
        
        inputs,targets = Variable(x), Variable(y)
        
        # Create new variables for the hidden state, otherwise we'd backprop
        # thourgh the entire training history
        ## Need to double check which options works better, detach or create new variables
        
        h = tuple([Variable(each.data) for each in h])
        
        net.zero_grad()
        
        output, h = net.forward(inputs,h)
        
        loss = criterion(output,targets.view(n_seqs*n_steps))
        
        print loss
        
        loss.backward()
        
        # clip_grad_norm helps prevent the exploding gradient
        # double check this
        nn.utils.clip_grad_norm(net.parameters(),clip)
        
        opt.step()
        
        if counter % print_every == 0:
            # Get validation loss
            val_h = net.init_hidden(n_seqs)
            val_losses = []
            for x, y in get_batches(val_data,n_seqs,n_steps):
                x = one_hot_encode(x,label_size)
                x,y = torch.from_numpy(x),torch.from_numpy(y)
                
                val_h = tuple([Variable(each.data,volatile= True) for each in val_h ])
                
                inputs, targets = Variable(x,volatile=True),Variable(y,volatile=True)
                
                output,val_h = net.forward(inputs,val_h)
                
                val_loss = criterion(output,targets.view(n_seqs*n_steps))
                
                val_losses.append(val_loss.data[0])
                
            print("Epoch: {}/{}...".format(e+1, epochs),
                  "Step: {}...".format(counter),
                  "Loss: {:.4f}...".format(loss.data[0]),
                  "Val Loss: {:.4f}".format(np.mean(val_losses)))
            

tensor(4.3960)


/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


tensor(3.8278)
tensor(3.6796)
tensor(3.4764)
tensor(3.4779)
tensor(3.4492)
tensor(3.3558)
tensor(3.3411)
tensor(3.3469)
tensor(3.2965)


/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:46: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:48: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:54: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/edwin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:58: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


('Epoch: 1/10...', 'Step: 10...', 'Loss: 3.2965...', 'Val Loss: 3.2920')
tensor(3.2986)
tensor(3.2863)
tensor(3.2664)
tensor(3.2839)
tensor(3.2500)
tensor(3.2333)
tensor(3.2243)
tensor(3.2257)
tensor(3.2096)
tensor(3.1942)
('Epoch: 1/10...', 'Step: 20...', 'Loss: 3.1942...', 'Val Loss: 3.1983')
tensor(3.1910)
tensor(3.1775)
tensor(3.1793)
tensor(3.1542)
tensor(3.1202)
tensor(3.1387)
tensor(3.1163)
tensor(3.1192)
tensor(3.1044)
tensor(3.0662)
('Epoch: 1/10...', 'Step: 30...', 'Loss: 3.0662...', 'Val Loss: 3.0821')
tensor(3.0642)
tensor(3.0456)
tensor(3.0302)
tensor(3.0290)
tensor(3.0011)
tensor(3.0034)
tensor(2.9824)
tensor(2.9437)
tensor(2.9210)
tensor(2.9130)
('Epoch: 1/10...', 'Step: 40...', 'Loss: 2.9130...', 'Val Loss: 2.9193')
tensor(2.9033)
tensor(2.8961)
tensor(2.8451)
tensor(2.8567)
tensor(2.8358)
tensor(2.7893)
tensor(2.7747)
tensor(2.7620)
tensor(2.7464)
tensor(2.7399)
('Epoch: 1/10...', 'Step: 50...', 'Loss: 2.7399...', 'Val Loss: 2.7589')
tensor(2.7378)
tensor(2.7304)
tenso

In [ ]:
def sample(net, size, prime='The', top_k=None, cuda=False):
        
    if cuda:
        net.cuda()
    else:
        net.cpu()

    net.eval()
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
size = 2000
prime='Anna'
top_k=5

In [ ]:
chars = [ch for ch in prime]
h = net.init_hidden(1)
for ch in prime:
    char, h = net.predict(ch, h, cuda=False, top_k=top_k)

In [ ]:
chars.append(char)

# Now pass in the previous character and get a new one
for ii in range(size):
    char, h = net.predict(chars[-1], h, cuda=False, top_k=top_k)
    chars.append(char)

print ''.join(chars)